# Items demand utility

Import all the needed modules

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np

Import from the statistics file the columns needed

In [2]:
excel_file = 'R12.xlsx'
df = pd.read_excel(excel_file, usecols = ['Item', 'Year month',
                                          'Sold qty', 'Used qty',
                                          'Not rep usage'])

Import from the master file the columns needed

In [3]:
master_file = 'Item master.xlsx'
df_master = pd.read_excel(master_file, usecols = ['Item no', 'Item name',
                                            'Product group descr'])
df_master['Item no'] = df_master['Item no'].astype(str)

Import supply data for suppliers

In [4]:
excel = 'Supply.xlsx'
df_supply = pd.read_excel(excel, usecols = ['Item no', 'Supplier'])
df_supply.drop_duplicates(inplace=True)
df_supply['Item no'] = df_supply['Item no'].astype(str)

Import Item Balance

In [5]:
excel_file = 'Item balance.xlsx'
df_balance = pd.read_excel(excel_file, usecols = ['Item no', 'Safety stock', 'On hand bal'])
df_balance['Item no'] = df_balance['Item no'].astype(str)

Get the first word of the item column using str.split and str.get function

In [6]:
df['Item no'] = df['Item'].str.split().str.get(0)
df['Item no'] = df['Item no'].astype(str)

Remove possible whitespaces in the figures

In [7]:
columns = ['Sold qty', 'Used qty', 'Not rep usage']

for column in columns:
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace(' ', '')
    df[column] = df[column].astype(float)

Get the demand

In [8]:
df['Demand'] = df['Sold qty'] + df['Used qty'] + np.where(df['Not rep usage'] > 0, df['Not rep usage'], 0)

Get the pivot table of the demand and year month and replace NaN with zeros

In [9]:
pivot_df = pd.pivot_table(df, values = 'Demand', index = 'Item no' , columns = 'Year month', aggfunc=np.sum)
pivot_df = pivot_df.fillna(0)
pivot_df = pd.DataFrame(pivot_df.to_records())
pivot_df.head()

,Item no,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,2001,2002,2003,2004,2005,2006
0,0645000,132.0,33.0,33.0,33.0,33.0,66.0,33.0,99.0,0.0,99.0,297.0,99.0,198.0,33.0,99.0,99.0,198.0,99.0
1,0645001,0.0,39.0,39.0,390.0,39.0,117.0,195.0,390.0,39.0,78.0,39.0,39.0,156.0,78.0,156.0,39.0,78.0,0.0
2,0645002,90.0,90.0,0.0,225.0,360.0,45.0,45.0,225.0,135.0,90.0,45.0,45.0,0.0,45.0,180.0,0.0,90.0,0.0
3,0645003,0.0,102.0,51.0,204.0,51.0,153.0,255.0,51.0,153.0,459.0,102.0,0.0,51.0,102.0,0.0,51.0,204.0,51.0
4,0645004,174.0,0.0,290.0,58.0,174.0,174.0,58.0,0.0,232.0,232.0,58.0,58.0,464.0,232.0,102.0,0.0,102.0,102.0


Total field for pivot_df

In [10]:
col_len = len(pivot_df.columns)
pivot_df['Total'] = pivot_df.iloc[:, 1:col_len].sum(axis=1, numeric_only=True)
pivot_df['Mean'] = pivot_df.iloc[:, 1:col_len].mean(axis=1, numeric_only=True)
pivot_df['StD'] = pivot_df.iloc[:, 1:col_len].std(axis=1, numeric_only=True) 
pivot_df['CV'] = pivot_df['StD'] / pivot_df['Mean']
pivot_df['Min'] = pivot_df.iloc[:, 1:col_len].min(axis=1, numeric_only=True)
pivot_df['Q25'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.25, axis=1, numeric_only=True).round(0)
pivot_df['Median'] = pivot_df.iloc[:, 1:col_len].median(axis=1, numeric_only=True)
pivot_df['Q67'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.67, axis=1, numeric_only=True).round(0)
pivot_df['Q75'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.75, axis=1, numeric_only=True).round(0)
pivot_df['Max'] = pivot_df.iloc[:, 1:col_len].max(axis=1, numeric_only=True)
pivot_df['Item no'] = pivot_df['Item no'].astype(str)
pivot_df.head()

,Item no,1901,1902,1903,1904,1905,1906,1907,1908,1909,...,Total,Mean,StD,CV,Min,Q25,Median,Q67,Q75,Max
0,0645000,132.0,33.0,33.0,33.0,33.0,66.0,33.0,99.0,0.0,...,1683.0,93.500000,75.294129,0.805285,0.0,33.0,99.0,99.0,99.0,297.0
1,0645001,0.0,39.0,39.0,390.0,39.0,117.0,195.0,390.0,39.0,...,1911.0,106.166667,116.467744,1.097027,0.0,39.0,58.5,93.0,146.0,390.0
2,0645002,90.0,90.0,0.0,225.0,360.0,45.0,45.0,225.0,135.0,...,1710.0,95.000000,97.483030,1.026137,0.0,45.0,67.5,90.0,124.0,360.0
3,0645003,0.0,102.0,51.0,204.0,51.0,153.0,255.0,51.0,153.0,...,2040.0,113.333333,114.113978,1.006888,0.0,51.0,76.5,122.0,153.0,459.0
4,0645004,174.0,0.0,290.0,58.0,174.0,174.0,58.0,0.0,232.0,...,2510.0,139.444444,120.381366,0.863293,0.0,58.0,102.0,174.0,218.0,464.0


Get the 'Item name' for each item. Merge it with the master_df

In [11]:
df_master = pd.merge(df_master, df_supply, on='Item no') # Suppliers

df_master_balance = pd.merge(df_master, df_balance, on='Item no')
df_master_balance['Item no'] = df_master_balance['Item no'].astype(str)

demand_df = pd.merge(df_master, pivot_df, on='Item no')
demand_df = demand_df.fillna(0)

demand_df_balance = pd.merge(df_master_balance, pivot_df, on='Item no')
demand_df_balance = demand_df_balance.fillna(0)

Output the demand to Excel

In [12]:
name = 'Items Demand.xlsx'
writer = pd.ExcelWriter(name)
demand_df.to_excel(writer, index = False)
writer.save()
print('Done bro!')

name = 'Items Demand Balance.xlsx'
writer = pd.ExcelWriter(name)
demand_df_balance.to_excel(writer, index = False)
writer.save()
print('Done bro!')

Done bro!
Done bro!


In [13]:
df_master_balance.head()

,Item no,Item name,Product group descr,Supplier,Safety stock,On hand bal
0,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,9
1,1334,TFSR 200 Black,203 Roof fans,S010 Systemair Sverige AB,3.0,6
2,1487,KT 50-25-4,202 Rectangular & Square Duct fans,S010 Systemair Sverige AB,0.0,1
3,1608,FK 125 Fast clamp,208 Accessories Ventilation,S670 Systemair GmbH,0.0,19
4,1611,FK 200 Fast clamp,208 Accessories Ventilation,S670 Systemair GmbH,0.0,4
